In [125]:
import json

#this won't work for you unless you import the data. See ReadMe for URL
with open('data/Austin/austin_texas_osm_line.geojson', encoding='utf8') as f:
    lines = json.load(f)

## Feed in the file
extract the coordinates for each line. We're keeping the lats and lons together in their own list

In [145]:
lat, lon = [], []

for i,feature in enumerate(lines['features']):
    line = feature['geometry']['coordinates']
    lats, lons = [], []
    for point in line:        
        if point[0] < 0:
            point[0], point[1] = point[1], point[0]
        lats.append(point[0])
        lons.append(point[1])
    lat.append(lats)
    lon.append(lons)
    if i == 15:
        break
        
lat[:5]

[[29.931372, 29.931369, 29.93138, 29.931381, 29.931381, 29.931389, 29.931391],
 [29.931381, 29.932068],
 [29.933104, 29.933062, 29.933056, 29.933064, 29.933083],
 [29.934565,
  29.93454,
  29.934526,
  29.934532,
  29.934549,
  29.934588,
  29.934641,
  29.934726,
  29.934821],
 [29.933061, 29.933058]]

### Split the lines into consecutive points

In [146]:
latlines = []
lonlines = []

for line in lat:
    latlines.extend([line[i],line[i+1]] for i in range(0,len(line)-1))

for line in lon:
    lonlines.extend([line[i],line[i+1]] for i in range(0,len(line)-1))
    
latlines[:5]

[[29.931372, 29.931369],
 [29.931369, 29.93138],
 [29.93138, 29.931381],
 [29.931381, 29.931381],
 [29.931381, 29.931389]]

### Get the average Lat/Lon
This will be used to get an average elevation for each line segement

In [147]:
avglat = []
avglon = []

for line in latlines:
    avglat.append(sum(line)/len(line))
for line in lonlines:
    avglon.append(sum(line)/len(line))

avglat[:5]

[29.9313705, 29.9313745, 29.9313805, 29.931381, 29.931385]

### Combine into a list of tuples

In [148]:
avg_coords = list(zip(avglat,avglon))
avg_coords[:5]

[(29.9313705, -98.2075124),
 (29.9313745, -98.2082114),
 (29.9313805, -98.20906790000001),
 (29.931381, -98.2094989),
 (29.931385, -98.20990889999999)]

### Get elevation
Convert tuples into 'locations' string for the Google Maps API in the form:
"29.11, -98.22 | 29.00, -98.00"

In [149]:
locations = str()
for i, coord in enumerate(avg_coords):
    locations+=(str(coord[0]) + ',' + str(coord[1]))
    if i < len(avg_coords) -1:
        locations += ' | '
    
locations

'29.9313705,-98.2075124 | 29.9313745,-98.2082114 | 29.9313805,-98.20906790000001 | 29.931381,-98.2094989 | 29.931385,-98.20990889999999 | 29.93139,-98.2102979 | 29.9317245,-98.2094009 | 29.933083,-98.2113244 | 29.933059,-98.2115084 | 29.93306,-98.2116134 | 29.9330735,-98.2117319 | 29.9345525,-98.2101954 | 29.934533000000002,-98.2104994 | 29.934529,-98.2107354 | 29.9345405,-98.21090989999999 | 29.9345685,-98.2110764 | 29.934614500000002,-98.21123639999999 | 29.9346835,-98.21140739999998 | 29.9347735,-98.2115944 | 29.9330595,-98.20809689999999 | 29.934542,-98.2077889 | 29.9359155,-98.21017289999999 | 29.9359365,-98.21036989999999 | 29.935966,-98.2104779 | 29.936051,-98.21070939999998 | 29.9352385,-98.2114219 | 29.9358855,-98.2110159 | 29.936591,-98.21058790000001 | 29.9373035,-98.2101514 | 29.9377245,-98.2098869 | 29.937949000000003,-98.2097434 | 29.938046,-98.2096674 | 29.938143,-98.20958390000001 | 29.9382455,-98.2094699 | 29.938333,-98.20935589999999 | 29.9383775,-98.2092794 | 29.9384

In [150]:
import requests
from YOUR_API_KEY import key

search_url = 'https://maps.googleapis.com/maps/api/elevation/json?'
search_payload = {"key": key, "locations": locations}
search_req = requests.get(search_url, params=search_payload)
search_json = search_req.json()

results = search_json['results'] 
results[:5]

INFO:requests.packages.urllib3.connectionpool:Starting new HTTPS connection (1): maps.googleapis.com


[{'elevation': 353.4602966308594,
  'location': {'lat': 29.9313705, 'lng': -98.2075124},
  'resolution': 9.543951988220215},
 {'elevation': 340.48681640625,
  'location': {'lat': 29.9313745, 'lng': -98.2082114},
  'resolution': 9.543951988220215},
 {'elevation': 337.2597961425781,
  'location': {'lat': 29.9313805, 'lng': -98.20906790000001},
  'resolution': 9.543951988220215},
 {'elevation': 333.9473571777344,
  'location': {'lat': 29.931381, 'lng': -98.2094989},
  'resolution': 9.543951988220215},
 {'elevation': 330.006591796875,
  'location': {'lat': 29.931385, 'lng': -98.20990889999999},
  'resolution': 9.543951988220215}]

### Get list of elevations

In [151]:
elevations = []

for i in results:
    elevations.append(i['elevation'])
elevations[:5]

[353.4602966308594,
 340.48681640625,
 337.2597961425781,
 333.9473571777344,
 330.006591796875]

In [154]:
from bokeh.io import show, output_file, output_notebook
from bokeh.plotting import figure

x=figure()
x.multi_line(latlines, lonlines)

output_file('test.html')
output_notebook()
show(x)

Loading BokehJS ...

ERROR:C:\Users\Kyle's Laptop\Anaconda3\lib\site-packages\bokeh\core\validation\check.py:E-1000 (COLUMN_LENGTHS): ColumnDataSource column lengths are not all the same: ColumnDataSource, ViewModel:ColumnDataSource, ref _id: 37b85a04-eb30-48e3-ab7a-9090c6a887c2


In [161]:
import pandas as pd

df = pd.MultiIndex(latlines, lonlines, columns = ['lat','lon'])

In [164]:
from bokeh.io import show, output_file, output_notebook
from bokeh.models import(GMapPlot, GMapOptions, ColumnDataSource,
                        Circle, DataRange1d, PanTool, WheelZoomTool, BoxSelectTool, )

In [165]:
map_options = GMapOptions(lat=30.29, lng=-97.73,
                         map_type='roadmap', zoom=14)

In [166]:
plot = GMapPlot(x_range=DataRange1d(),
                y_range=DataRange1d(),
                map_options=map_options)
plot.title.text = "Austin"
plot.api_key=key
plot.add_tools(PanTool(), WheelZoomTool(), BoxSelectTool())

In [169]:
baseline = 0
scale = 5

source = ColumnDataSource(
    data=dict(
        lat=[30.29, 30.20, 30.29],
        lon=[-97.70, -97.74, -97.78]
        #rad = [(i-baseline) / scale for i in df.index.tolist()]
    )
)

In [170]:
from bokeh.io import show, output_file
from bokeh.plotting import figure

# [[30.39, 30.10, 30.19],[30.29, 30.20, 30.29]], [[-97.80, -97.84, -97.88],[-97.70, -97.74, -97.78]]
p=figure()
p.multi_line(lat, lon, color=['firebrick','navy','red','blue'])
#        color=["firebrick", "navy"], alpha=[0.8, 0.3], line_width=1.5) #fill_alpha=0.5,
output_file('line.html')
show(p)

ERROR:C:\Users\Kyle's Laptop\Anaconda3\lib\site-packages\bokeh\core\validation\check.py:E-1000 (COLUMN_LENGTHS): ColumnDataSource column lengths are not all the same: ColumnDataSource, ViewModel:ColumnDataSource, ref _id: 37b85a04-eb30-48e3-ab7a-9090c6a887c2
ERROR:C:\Users\Kyle's Laptop\Anaconda3\lib\site-packages\bokeh\core\validation\check.py:E-1000 (COLUMN_LENGTHS): ColumnDataSource column lengths are not all the same: ColumnDataSource, ViewModel:ColumnDataSource, ref _id: 204ef8f5-f61f-4a07-98cf-012808165709


In [171]:
circle = Circle(x='lon', y='lat',size=2,
                fill_color='blue',fill_alpha=0.5)

plot.add_glyph(source, circle)

In [55]:
output_file('basicplot.html')
# output_notebook()
show(plot)
show(p)

INFO:bokeh.core.state:Session output file 'basicplot.html' already exists, will be overwritten.
